# AttackEval class for evaluating the performance on defaults scenarios

Example of usage of the TransferBench library to evaluate the attack performance of on an adversarial attacks on preset or custom scenarios.

In [ ]:
%cd ..

# case 1: Evaluate Attack using a default scenario

In [ ]:
from transferbench.attack_evaluation import AttackEval
from transferbench.attacks_zoo import NaiveAvg

evaluator = AttackEval(NaiveAvg)
# Default scenarios are:
print(evaluator.scenarios)
# Running the evaluation
result = evaluator.run(batch_size=4, device="cuda:1")
print(result)

# Case 2: Evaluate Attack using custom scenarios, on torchvision models

In [ ]:
from transferbench.attack_evaluation import AttackEval
from transferbench.attacks_zoo import NaiveAvg
from transferbench.scenarios import list_scenarios

evaluator = AttackEval(NaiveAvg)
# Default scenarios are:
print(list_scenarios())
evaluator.set_scenarios("homo-imagenet-inf")
result = evaluator.run(batch_size=4, device="cuda:1")


# Case 3

### Evaluating on a personilized scenarios with custom models and datasets

In [ ]:
from transferbench.attack_evaluation import AttackEval
from transferbench.attacks_zoo import NaiveAvg
from transferbench.scenarios import AttackScenario
from torchvision import transforms

# Load a dataset
transform = transforms.Compose([transforms.ToTensor()])
cifar100 = datasets.CIFAR100(
    root="./data/datasets",
    train=False,
    download=True,
    transform=transform,
)
cifar100_mean = [0.5, 0.5, 0.5]
cifar100_std = [1.0, 1.0, 1.0]

REPO_LINK = "chenyaofo/pytorch-cifar-models"
# Load models and normalize them
def get_model(model):
    return torch.hub.load(
    REPO_LINK, "cifar100_" + model, pretrained=True
)

# Use the dataset in the TransferEval
victim_model = add_normalization(get_model("resnet56"), cifar100_mean, cifar100_std)
surrgoate_models = [
    add_normalization(get_model("vgg11_bn"), cifar100_mean, cifar100_std),
    add_normalization(get_model("vgg13_bn"), cifar100_mean, cifar100_std),
    add_normalization(get_model("vgg16_bn"), cifar100_mean, cifar100_std),
    add_normalization(get_model("vgg19_bn"), cifar100_mean, cifar100_std),
]

evaluator = TransferEval(
    victim_model,
    surrgoate_models,
)
myscenario = TransferScenario(
    hp=BaseHyperParameters(eps=0.3, p=2, maximum_queries=10),
    attack_step="NaiveAvg",
    dataset=cifar100,
)
evaluator.set_scenarios(myscenario)
result = evaluator.run(batch_size=4, device="cuda:1")
print(result)
